In [ ]:
import scanpy as sc
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd
import scvelo as scv
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import matplotlib.pyplot as pl
import igraph
import loompy as lmp
import anndata
from scipy.sparse import coo_matrix, csr_matrix

In [ ]:
# Scvelo Setting
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
# get the path of the loom files
directory_path = '/groups/mb928_gp/adr2189/velocyto_visium/loom'
file_names = []

In [ ]:
# Get the file names
for file_name in os.listdir(directory_path):
    if file_name.endswith('.loom'):
        file_path = os.path.join(directory_path, file_name)
        file_names.append(file_path)

In [ ]:
from natsort import natsorted
file_names = natsorted(file_names, key=lambda p: p.split("loom/")[1])
file_names

In [ ]:
# Function to concatenate loom files
def load_and_concatenate_loom_files(filenames):
    ldata_list = []

    for filename in file_names:
        ldata = sc.read(filename, cache=True)
        sample_id = filename.split("loom/")[1][:5]
        barcodes = [bc.split(':')[1] for bc in ldata.obs.index.tolist()]
        barcodes = [sample_id + '_' + bc for bc in barcodes]
        ldata.obs.index = barcodes
        ldata.var_names_make_unique()
        ldata_list.append(ldata)

    ldata = ldata_list[0].concatenate(ldata_list[1:], join='outer', batch_key=None, index_unique="")

    return ldata

In [ ]:
ldata = load_and_concatenate_loom_files(file_names)

In [ ]:
#standardized index for ldata to match the adata
ldata.obs.index = [idx[:22] + '-1' for idx in ldata.obs.index]

In [ ]:
adata = sc.read('/groups/mb928_gp/ts3593/rna_velovity/v1/v1.h5ad')

In [ ]:
#Combine ldata and adata
common_obs = ldata.obs_names.intersection(adata.obs_names)
common_vars = ldata.var_names.intersection(adata.var_names)

In [ ]:
ldata = ldata[common_obs].copy()

In [ ]:
adata = scv.utils.merge(adata, ldata)

In [ ]:
#pre-processing
sc.pp.pca(
    adata,
    n_comps=40,            # match Seurat's npcs
    svd_solver='arpack',   # deterministic, like Seurat's truncated SVD
    random_state=42
)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30)
scv.pp.moments(adata)

In [ ]:
#run dynamical model
scv.tl.recover_dynamics(adata)
adata.write(new_v1_full.h5ad)

In [ ]:
#get the velocity map
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', color = "seurat_clusters")

In [ ]:
#umap
sc.pl.umap(adata, color = "seurat_clusters", frameon=False, legend_loc = "on data", save = "umap.pdf")

In [ ]:
#get the subdataset only from gcl, sgz.pl and sgz.ml
adata_sub = adata_origin[adata_origin.obs['seurat_clusters'].isin(['gcl','sgz.pl','sgz.ml'])].copy()

In [ ]:
#pre-processing
scv.pp.filter_and_normalize(adata_sub, retain_genes = Neurogenesis, flavor= "seurat", n_top_genes = 2000)
sc.pp.pca(
    adata_sub,
    n_comps=40,            # match Seurat's npcs
    svd_solver='arpack',   # deterministic, like Seurat's truncated SVD
    random_state=42
)
sc.pp.neighbors(adata_sub, n_pcs=30, n_neighbors=30)
scv.pp.moments(adata_sub)

In [ ]:
#subdata run dynamical model
scv.tl.recover_dynamics(adata_sub)

In [ ]:
#velocity
scv.tl.velocity(adata_sub, mode='dynamical')
scv.tl.velocity_graph(adata_sub)
scv.pl.velocity_embedding_stream(adata_sub, basis='umap', color = "seurat_clusters")

In [ ]:
#latent time
scv.tl.latent_time(adata_sub)
scv.pl.scatter(adata_sub, color='latent_time', color_map='gnuplot', save = "sub_latent_time.png")

In [ ]:
#paga
adata_sub.uns['neighbors']['distances'] = adata_sub.obsp['distances']
adata_sub.uns['neighbors']['connectivities'] = adata_sub.obsp['connectivities']
scv.tl.paga(adata_sub, groups='seurat_clusters')
scv.pl.paga(adata_sub, basis='umap', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1.5, save = "sub_paga.pdf")

In [ ]:
#get the top genes for the heatmap
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:200]

In [ ]:
#get the heatmap
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:200]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time',yticklabels=True,figsize=(11,30), col_color='seurat_clusters', n_convolve=100,save = 'latenttime_heatmap_top_200_genes.pdf')

In [ ]:
#Neurogensis that are interested
Neurogenesis = ['MARCKSL1', 'PCNA', 'BMPR2', 'GM11837', 'BHLHE22', 'CDK4', 'NEUROD1', 'TUBB3', 
                'C530008M17RIK', 'MYL6B', 'TRAF4', 'TUBB5', 'SMIM18', 'FST', 'OST4', 'RBFOX3',
                'CASR', 'CRIP2', 'CRABP1', 'BMP2', 'HBA-A1', 'SHH', 'STMN2', 'HOXA9', 'MEX3A',
                'DLX2', 'MKI67', 'CYP27C1', 'IDH1', 'CHRM3', 'SMAD7', 'IFIT3', 'DVL1', 'SV2B',
                'TUBB2A', 'FAM166C', 'ETNPPL', 'EPS8', 'GM17750', 'CSDC2', 'SOX2', 'NR2E1',
                'OR2V1', 'BMPR1B', 'MEIS2', 'RGS6', 'ZFP57', 'GDF7', 'CD24A', 'BMP7', 'WNT7A',
                'SNCG', 'RSPO2', 'NOTCH1', 'MYT1L', 'PRDX1', 'SOX11', 'HAPLN4', 'INSM1', 'CRLF1',
                'NEUROD6', 'SOCS2', 'WNT3A', 'COX8A', 'RND2', 'BACH2', 'MAPK6', 'SEMA3D', 'DNER',
                'ARHGDIG', 'FXYD7', 'SCRG1', 'PTCH1', 'SSTR2', 'SYT1', 'CXADR', 'GM10320', 'SMO',
                'PROX1', 'PGAP1', 'NANS', 'MTSS1', 'NEUROD2', 'ARL4C', 'TOP2A', 'SFRP2', 'PCDH15',
                'ST8SIA2', 'NELL1', 'COL12A1', 'GLI2', 'MCM2', 'MAD2L2', 'DPYSL5', 'RELN', 'AXIN2',
                'HSPB8', 'POSTN', 'TAC1', 'OLIG1', 'APLN', 'HOXD13', 'EMC7', 'LMX1B', 'DCX',
                'AEBP1', 'FNBP1L', 'TBR1', 'PGLS', 'TIMP1', 'OTX2', 'PRKRA', 'TUBA1A', 'CARHSP1',
                'NMB', 'LAMA2', 'TMSB4X', 'LMX1A', 'ANGPT1', 'EPHA3', 'FGFR1', 'CNTNAP2', 'TIMM13',
                'FXYD6', 'EOMES', 'PPP1R14C', 'CALB1', 'VIM', 'CHMP2A', 'LAMP5', 'CAMK4', 'HOXA10',
                'IER2', 'L1CAM', 'S100A6', 'BMPR1A', 'NNAT', 'KIF21B', 'SH3YL1', 'SHD', 'TAGLN2',
                'RERG', 'PAFAH1B3', 'TTC28', 'CKS1B', 'TUBB2B', 'HSPA2', 'HEPN1', 'ESM1', 'CSRP2',
                'S100B', 'H2AFY2', 'SYCE2', 'HES5', 'STC1', 'GAD2', 'ELMO1', 'IGFBPL1', 'WNT5A',
                'SOX4', 'TRIM67', 'NOG', 'CIB1', 'PAX6', 'CDC20B', 'ID3', 'CTNNB1', 'GAP43', 'HOXC6',
                'HHIP', 'FABP7', 'HEY2', 'MBP', 'GM9844', 'LGALS1', 'FOS', 'ASCL1', 'CALB2', 'BMP4',
                'PRDX5', 'ATAT1', 'B2M', 'TPPP3', 'HCN4', 'SQLE', 'AQP4', 'SMILR', 'RBP1', 'STMN1',
                'GAD1', 'ELAVL2', 'DVL2', 'DRAXIN', 'CADM4', 'HOXB4', 'FABP5', 'CETN2', 'LRP6',
                'HBB-BS', 'RMST', 'FZD7', 'GREM1', 'TRPC6', 'NOTCH2', 'NES', 'RAC3', 'HMGB3', 'DPYSL3',
                'GLI1', 'GFAP', 'OTOF', 'S100A10', 'CENPW', 'SBK1', 'NEUROG3', 'ADAMTS18', 'NEUROG2',
                'COL8A2', 'FEZF2', 'GSTP1', 'MIDN', 'FRMD3', 'SEMA3C', 'AFAP1L2', 'TMSB10', 'FOXO3', 'RNF121', 'OLIG2', 'NDUFC1', 'DDAH2', 'DAP', 'SSNA1']

In [ ]:
present = [g for g in top_genes if g in Neurogenesis]
present

In [ ]:
#Detailed pot for genes
for gene in new_present:
    scv.pl.velocity(adata, gene, color = "seurat_clusters", dpi = 72)